<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/temporal-analysis/completely_home/analyse_LA_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 2.03 ms (started: 2021-04-20 23:30:52 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 44.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f04328b97fe6357f6fb389c67879c69280f5906b305d9fb8894bd78fdc040306
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 100 ms (started: 2021-04-20 23:31:44 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 291 ms (started: 2021-04-20 23:31:44 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.66 s (started: 2021-04-20 23:31:44 +00:00)


In [6]:
#sc.stop()

time: 1.49 ms (started: 2021-04-20 23:31:49 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 16.1 s (started: 2021-04-20 23:31:49 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 2.34 ms (started: 2021-04-20 23:32:05 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|                date|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|
|060374034022|2020-02-07T00:00:...|         155|                          24|
|060375326041|2020-02-07T00:00:...|          31|                           4|
|060375544031|2020-02-07T00:00:...|         130|                          27|
|060375718003|2020-02-07T00:00:...|          35|                          10|
|060590018024|2020-02-07T00:00:...|          38|                          13|
|060590320463|2020-02-07T00:00:...|         184|                          24|
|060590626281|2020-02-07T00:00:...|         108|                          32|
|060371374011|2020-02-07T00:00:...|          72|                          30|
|060372398012|2020-02-07T00:00:...|          41|                

In [11]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 45.9 ms (started: 2021-04-20 23:32:33 +00:00)


In [12]:
df_soc.show()

+------------+----------+------------+----------------------------+
|         cbg|      date|device_count|completely_home_device_count|
+------------+----------+------------+----------------------------+
|060371831013|2020-02-07|          45|                           6|
|060374034022|2020-02-07|         155|                          24|
|060375326041|2020-02-07|          31|                           4|
|060375544031|2020-02-07|         130|                          27|
|060375718003|2020-02-07|          35|                          10|
|060590018024|2020-02-07|          38|                          13|
|060590320463|2020-02-07|         184|                          24|
|060590626281|2020-02-07|         108|                          32|
|060371374011|2020-02-07|          72|                          30|
|060372398012|2020-02-07|          41|                          15|
|060373009011|2020-02-07|          82|                          20|
|060375418021|2020-02-07|          49|          

In [14]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 60.9 ms (started: 2021-04-20 23:32:52 +00:00)


In [15]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 37.1 ms (started: 2021-04-20 23:33:00 +00:00)




---


**Demographic data**





---



In [16]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [17]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+----------------------------+--------------------------+------------------+
|         cbg|      date|device_count|completely_home_device_count|completely_home_percentage|poverty_percentage|
+------------+----------+------------+----------------------------+--------------------------+------------------+
|060371831013|2020-02-07|          45|                           6|        13.333333333333334|15.450643776824036|
|060374034022|2020-02-07|         155|                          24|        15.483870967741936| 10.67193675889328|
|060375326041|2020-02-07|          31|                           4|        12.903225806451612| 27.79187817258883|
|060375544031|2020-02-07|         130|                          27|         20.76923076923077|  25.3447633246366|
|060375718003|2020-02-07|          35|                          10|         28.57142857142857|3.1537450722733245|
|060590018024|2020-02-07|          38|                          13|         34.210526315

In [34]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as top20_perc,
l.completely_home_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 283 ms (started: 2021-04-21 00:07:50 +00:00)


**Load 2019 stats**

In [35]:
# path_str = 'drive/MyDrive/big-data-project/sketch/justin/{}_social_summary_'.format(city)
# path = Path(path_str)

# stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/'2019.csv'))
# stat_2019

mean_2019 = 30.14549191952775

time: 1.8 ms (started: 2021-04-21 00:07:53 +00:00)


In [41]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 35.5 ms (started: 2021-04-21 00:13:32 +00:00)


In [42]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 23.4 ms (started: 2021-04-21 00:13:37 +00:00)



**Visualizations** 



In [301]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#E15759', '#a0aab4', '#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

ThemeRegistry.enable('my_theme')

time: 13.5 ms (started: 2021-04-21 06:16:11 +00:00)


In [391]:
def create_chart(df) :
  chart = alt.Chart(df1).transform_fold(
    #fold=['bottom20_deviation', 'top20_deviation','mean 2019'],
    fold=['bottom20_deviation', 'top20_deviation'],
    as_ = ['Poverty %','completely home %']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('completely home %:Q', title = 'Completely Home %'),
      color='Poverty %:N'
    ).properties(width = 1000)

  line = alt.Chart(df1).mark_line(color='grey').encode(x='date:T',y=alt.Y('mean 2019'))

  return chart + line


time: 7.76 ms (started: 2021-04-21 07:21:16 +00:00)


In [345]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 1.89 ms (started: 2021-04-21 06:41:53 +00:00)


In [304]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,17.297551,17.028372,-12.847941,-13.117120
1,2020-04-13,49.179507,27.580760,19.034015,-2.564731
2,2020-06-24,33.002194,36.514949,2.856702,6.369457
3,2020-06-08,36.723946,40.887284,6.578454,10.741793
4,2020-09-12,33.388659,29.334402,3.243167,-0.811090
...,...,...,...,...,...
361,2020-04-05,51.744280,27.824742,21.598788,-2.320750
362,2020-05-01,43.087875,43.937332,12.942383,13.791840
363,2020-10-25,35.361961,32.028754,5.216469,1.883262
364,2020-12-28,41.489642,24.049980,11.344151,-6.095512


time: 1.72 s (started: 2021-04-21 06:16:17 +00:00)


In [305]:
# add the mean column 2019
df['mean 2019'] = 0

time: 1.68 ms (started: 2021-04-21 06:16:22 +00:00)


In [385]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['bottom20_perc', 'top20_perc'],
    as_ = ['poverty %','completely home %']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('completely home %:Q', title = 'Completely Home %'),      color='poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 48.5 ms (started: 2021-04-21 07:19:37 +00:00)


In [340]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 8.3 ms (started: 2021-04-21 06:39:06 +00:00)


Filtering based on important dates



*   Thursday, March 19, 2020: Statewide Stay-at-Home Order Issued











In [392]:
start_date = '2020-3-16'
end_date = '2020-4-9'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 36.8 ms (started: 2021-04-21 07:21:19 +00:00)





*   August. 7, 2020: LA County Exceeds 200,000 Coronavirus Cases
*   August, 12, 2020: LA County Crosses 'Tragic Milestone' Of 5,000 Coronavirus Deaths 









In [393]:
start_date = '2020-8-5'
end_date = '2020-8-21'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart

alt.LayerChart(...)

time: 43.3 ms (started: 2021-04-21 07:21:24 +00:00)


**Oct. 14, 2020: LA County Sees Rise In Workplace Outbreaks As Infection Rate Creeps Upward**

--- 

In [394]:
start_date = '2020-10-12'
end_date = '2020-10-24'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 52.1 ms (started: 2021-04-21 07:21:27 +00:00)


Dec. 16, 2020:
LA County COVID-19 Deaths Hit New Record

---

In [395]:
start_date = '2020-12-15'
end_date = '2020-12-31'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 30 ms (started: 2021-04-21 07:21:30 +00:00)
